<a href="https://colab.research.google.com/github/danieldai/Dive-into-DL-PyTorch-in-Colab/blob/main/3_6_softmax%E5%9B%9E%E5%BD%92%E7%9A%84%E4%BB%8E%E9%9B%B6%E5%BC%80%E5%A7%8B%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://tangshusen.me/Dive-into-DL-PyTorch/#/chapter03_DL-basics/3.6_softmax-regression-scratch

下载 d2lzh_pytorch 到colab环境中

In [2]:
!git clone https://github.com/ShusenTang/Dive-into-DL-PyTorch.git
#!ls Dive-into-DL-PyTorch/code/d2lzh_pytorch
!cp -r Dive-into-DL-PyTorch/code/d2lzh_pytorch /content

Cloning into 'Dive-into-DL-PyTorch'...
remote: Enumerating objects: 1822, done.
remote: Total 1822 (delta 0), reused 0 (delta 0), pack-reused 1822
Receiving objects: 100% (1822/1822), 34.33 MiB | 17.07 MiB/s, done.
Resolving deltas: 100% (1058/1058), done.


# 3.6 softmax回归的从零开始实现

这一节我们来动手实现softmax回归。首先导入本节实现所需的包或模块。

In [3]:
import torchtext
torchtext.disable_torchtext_deprecation_warning()

import torch
import torchvision
import numpy as np
import sys
sys.path.append("..") # 为了导入上层目录的d2lzh_pytorch
import d2lzh_pytorch as d2l



## 3.6.1 获取和读取数据

我们将使用Fashion-MNIST数据集，并设置批量大小为256。

In [4]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, root='/content/Datasets/FashionMNIS')


100%|██████████| 26421880/26421880 [00:02<00:00, 11393501.30it/s]


Extracting /content/Datasets/FashionMNIS/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/Datasets/FashionMNIS/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 190853.87it/s]


Extracting /content/Datasets/FashionMNIS/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/Datasets/FashionMNIS/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3647137.18it/s]


Extracting /content/Datasets/FashionMNIS/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/Datasets/FashionMNIS/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 18267577.83it/s]

Extracting /content/Datasets/FashionMNIS/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/Datasets/FashionMNIS/FashionMNIST/raw




/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## 3.6.2 初始化模型参数
跟线性回归中的例子一样，我们将使用向量表示每个样本。已知每个样本输入是高和宽均为28像素的图像。模型的输入向量的长度是 $28 \times 28 = 784$：该向量的每个元素对应图像中每个像素。由于图像有10个类别，单层神经网络输出层的输出个数为10，因此softmax回归的权重和偏差参数分别为$784 \times 10$和$1 \times 10$的矩阵。

In [5]:
num_inputs = 784
num_outputs = 10

W = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)


同之前一样，我们需要模型参数梯度。

In [6]:
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

## 3.6.3 实现softmax运算

在介绍如何定义softmax回归之前，我们先描述一下对如何对多维`Tensor`按维度操作。在下面的例子中，给定一个`Tensor`矩阵`X`。我们可以只对其中同一列（`dim=0`）或同一行（`dim=1`）的元素求和，并在结果中保留行和列这两个维度（`keepdim=True`）。

In [7]:
X = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(X.sum(dim=0, keepdim=True))
print(X.sum(dim=1, keepdim=True))

tensor([[5, 7, 9]])
tensor([[ 6],
        [15]])


下面我们就可以定义前面小节里介绍的softmax运算了。在下面的函数中，矩阵`X`的行数是样本数，列数是输出个数。为了表达样本预测各个输出的概率，softmax运算会先通过`exp`函数对每个元素做指数运算，再对`exp`矩阵同行元素求和，最后令矩阵每行各元素与该行元素之和相除。这样一来，最终得到的矩阵每行元素和为1且非负。因此，该矩阵每行都是合法的概率分布。softmax运算的输出矩阵中的任意一行元素代表了一个样本在各个输出类别上的预测概率。

In [ ]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(dim=1, keepdim=True)
    return X_exp / partition  # 这里应用了广播机制


In [ ]:
def net(X):
    return softmax(torch.mm(X.view((-1, num_inputs)), W) + b)


In [ ]:
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])
y_hat.gather(1, y.view(-1, 1))


tensor([[0.1000],
        [0.5000]])

In [ ]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat.gather(1, y.view(-1, 1)))


In [ ]:
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()


In [ ]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用。该函数将被逐步改进：它的完整实现将在“图像增广”一节中描述
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n


In [ ]:
num_epochs, lr = 5, 0.1

# 本函数已保存在d2lzh包中方便以后使用
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()

            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()

            l.backward()
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                optimizer.step()  # “softmax回归的简洁实现”一节将用到


            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)


epoch 1, loss 0.7840, train acc 0.748, test acc 0.791
epoch 2, loss 0.5708, train acc 0.812, test acc 0.811
epoch 3, loss 0.5257, train acc 0.826, test acc 0.815
epoch 4, loss 0.5018, train acc 0.832, test acc 0.824
epoch 5, loss 0.4858, train acc 0.836, test acc 0.829


In [ ]:
X, y = iter(test_iter).next()

true_labels = d2l.get_fashion_mnist_labels(y.numpy())
pred_labels = d2l.get_fashion_mnist_labels(net(X).argmax(dim=1).numpy())
titles = [true + '\n' + pred for true, pred in zip(true_labels, pred_labels)]

d2l.show_fashion_mnist(X[0:9], titles[0:9])


AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'